In [0]:
df_cust = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/202211037@daiict.ac.in/US_Regional_Sales_Data_Customers.csv")
df_loc = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/202211037@daiict.ac.in/US_Regional_Sales_Data_Store_Locations.csv")
df_prod = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/202211037@daiict.ac.in/US_Regional_Sales_Data_Products.csv")
df_reg = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/202211037@daiict.ac.in/US_Regional_Sales_Data_Regions.csv")
df_sales_team = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/202211037@daiict.ac.in/US_Regional_Sales_Data_Sales_Team.csv")
df_sales_ord = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/202211037@daiict.ac.in/US_Regional_Sales_Data_Sales_Order.csv")

In [0]:
df_cust.show(10)

+-----------+--------------+
|_CustomerID|Customer Names|
+-----------+--------------+
|          1|     Avon Corp|
|          2|     WakeFern |
|          3|  Elorac, Corp|
|          4|     ETUDE Ltd|
|          5|  Procter Corp|
|          6| PEDIFIX, Corp|
|          7|       New Ltd|
|          8|  Medsep Group|
|          9|           Ei |
|         10|      21st Ltd|
+-----------+--------------+
only showing top 10 rows



In [0]:
df_loc.show(10)

+--------+-----------+--------------------+---------+--------+----+--------+----------+--------+----------+----------------+-------------+---------+----------+---------------+
|_StoreID|  City Name|              County|StateCode|   State|Type|Latitude| Longitude|AreaCode|Population|Household Income|Median Income|Land Area|Water Area|      Time Zone|
+--------+-----------+--------------------+---------+--------+----+--------+----------+--------+----------+----------------+-------------+---------+----------+---------------+
|       1| Birmingham|Shelby County/Jef...|       AL| Alabama|City|33.52744| -86.79905|     205|    212461|           89972|        31061|378353942|   6591013|America/Chicago|
|       2| Huntsville|Limestone County/...|       AL| Alabama|City|34.69901| -86.67298|     256|    190582|           78554|        48775|552604579|   3452021|America/Chicago|
|       3|     Mobile|       Mobile County|       AL| Alabama|City|30.69436| -88.04305|     251|    194288|           76

In [0]:
df_prod.show(10)

+----------+------------------+
|_ProductID|      Product Name|
+----------+------------------+
|         1|          Cookware|
|         2|      Photo Frames|
|         3|       Table Lamps|
|         4|         Serveware|
|         5|Bathroom Furniture|
|         6|         Computers|
|         7|        Dinnerware|
|         8|  Cocktail Glasses|
|         9|          Baseball|
|        10|          Blankets|
+----------+------------------+
only showing top 10 rows



In [0]:
df_reg.show(10)

+---------+--------------------+---------+
|StateCode|               State|   Region|
+---------+--------------------+---------+
|       AL|             Alabama|    South|
|       AR|            Arkansas|    South|
|       AZ|             Arizona|     West|
|       CA|          California|     West|
|       CO|            Colorado|     West|
|       CT|         Connecticut|Northeast|
|       DC|District of Columbia|    South|
|       DE|            Delaware|    South|
|       FL|             Florida|    South|
|       GA|             Georgia|    South|
+---------+--------------------+---------+
only showing top 10 rows



In [0]:
df_sales_team.show(10)

+------------+----------------+---------+
|_SalesTeamID|      Sales Team|   Region|
+------------+----------------+---------+
|           1|  Adam Hernandez|Northeast|
|           2|   Keith Griffin|Northeast|
|           3|     Jerry Green|     West|
|           4| Chris Armstrong|Northeast|
|           5|   Stephen Payne|    South|
|           6|  Joshua Bennett|Northeast|
|           7|      Shawn Cook|  Midwest|
|           8|    George Lewis|     West|
|           9|     Joshua Ryan|  Midwest|
|          10|Jonathan Hawkins|     West|
+------------+----------------+---------+
only showing top 10 rows



In [0]:
df_sales_ord.show(10)

+-----------+-------------+-------------+------------+---------+---------+------------+------------+------------+-----------+--------+----------+--------------+----------------+----------+---------+
|OrderNumber|Sales Channel|WarehouseCode|ProcuredDate|OrderDate| ShipDate|DeliveryDate|CurrencyCode|_SalesTeamID|_CustomerID|_StoreID|_ProductID|Order Quantity|Discount Applied|Unit Price|Unit Cost|
+-----------+-------------+-------------+------------+---------+---------+------------+------------+------------+-----------+--------+----------+--------------+----------------+----------+---------+
|SO - 000101|     In-Store| WARE-UHY1004|  12/31/2017|5/31/2018|6/14/2018|   6/19/2018|         USD|           6|         15|     259|        12|             5|           0.075|  1,963.10| 1,001.18|
|SO - 000102|       Online| WARE-NMK1003|  12/31/2017|5/31/2018|6/22/2018|    7/2/2018|         USD|          14|         20|     196|        27|             3|           0.075|  3,939.60| 3,348.66|
|SO -

##1. Compute top-10 selling products in terms of numbers (i. e. sum(qty) )

In [0]:
from pyspark.sql import SparkSession
 
# Create a SparkSession
spark = SparkSession.builder.appName("ProductQuantity").getOrCreate()
 
# Register the DataFrames as temporary tables to use in SQL queries
df_prod.createOrReplaceTempView("df_prod")
df_sales_ord.createOrReplaceTempView("df_sales_ord")
 
# Write the SQL query using backticks to reference column names with spaces
query = """
SELECT p.`Product Name`, SUM(s.`Order Quantity`) AS Total_Quantity
FROM df_prod p
JOIN df_sales_ord s
ON p.`_ProductID` = s.`_ProductID`
GROUP BY p.`Product Name`
ORDER BY Total_Quantity DESC
LIMIT 10
"""
 
# Execute the SQL query
result = spark.sql(query)
 
# Show the result
result.show()

+----------------+--------------+
|    Product Name|Total_Quantity|
+----------------+--------------+
|     Accessories|         956.0|
|        Platters|         896.0|
|Cocktail Glasses|         879.0|
|       Serveware|         878.0|
|            Rugs|         855.0|
|    Collectibles|         854.0|
|    Wine Storage|         837.0|
|       Wardrobes|         832.0|
|         Wreaths|         830.0|
|Dining Furniture|         827.0|
+----------------+--------------+



In [0]:
from pyspark.sql import functions as F

In [0]:
df_prod.join(df_sales_ord, df_prod["_ProductID"] == df_sales_ord["_ProductID"])\
         .groupBy(df_prod["Product Name"])\
         .agg(F.sum(df_sales_ord["Order Quantity"]).alias("Total_Quantity"))\
         .orderBy(F.desc("Total_Quantity"))\
         .show(10)

+----------------+--------------+
|    Product Name|Total_Quantity|
+----------------+--------------+
|     Accessories|         956.0|
|        Platters|         896.0|
|Cocktail Glasses|         879.0|
|       Serveware|         878.0|
|            Rugs|         855.0|
|    Collectibles|         854.0|
|    Wine Storage|         837.0|
|       Wardrobes|         832.0|
|         Wreaths|         830.0|
|Dining Furniture|         827.0|
+----------------+--------------+
only showing top 10 rows



## 2. Compute top-10 selling products in terms of value (i. e. sum (qty*price) )

In [0]:
from pyspark.sql import SparkSession

In [0]:
# Create a SparkSession
spark = SparkSession.builder.appName("ProductGain").getOrCreate()
 
# Register the DataFrames as temporary tables to use in SQL queries
df_prod.createOrReplaceTempView("df_prod")
df_sales_ord.createOrReplaceTempView("df_sales_ord")
 
# Write the SQL query using backticks to reference column names with spaces
q = """ SELECT p.`Product Name`, SUM(s.`Order Quantity` * s.`Unit Price`) AS TotalGain
FROM df_prod p
JOIN df_sales_ord s
ON p.`_ProductID` = s.`_ProductID`
GROUP BY p.`Product Name`
ORDER BY TotalGain DESC
LIMIT 10 """
 
# Execute the SQL query
result = spark.sql(q)
 
# Show the result
result.show()

+----------------+------------------+
|    Product Name|         TotalGain|
+----------------+------------------+
|        Pendants|          150240.8|
|     Accessories|          133376.9|
|Cocktail Glasses|124492.70000000001|
|          Phones|124037.10000000002|
|        Platters|          122744.0|
|     Table Lamps|          119172.9|
| Home Fragrances|113960.29999999999|
|    Collectibles|111796.20000000003|
|           Vases|          110020.7|
|        Bakeware|108399.29999999999|
+----------------+------------------+



In [0]:
from pyspark.sql import functions as F

In [0]:
df_prod.join(df_sales_ord, df_prod["_ProductID"] == df_sales_ord["_ProductID"])\
         .groupBy(df_prod["Product Name"])\
         .agg(F.sum(df_sales_ord["Order Quantity"]*df_sales_ord["Unit Price"]).alias("Total_Revenue"))\
         .orderBy(F.desc("Total_Revenue"))\
         .show(10)

+----------------+------------------+
|    Product Name|     Total_Revenue|
+----------------+------------------+
|        Pendants|          150240.8|
|     Accessories|          133376.9|
|Cocktail Glasses|124492.70000000001|
|          Phones|124037.10000000002|
|        Platters|          122744.0|
|     Table Lamps|          119172.9|
| Home Fragrances|113960.29999999999|
|    Collectibles|111796.20000000003|
|           Vases|          110020.7|
|        Bakeware|108399.29999999999|
+----------------+------------------+
only showing top 10 rows



## 3. Compute top-10 profit making products. Profit = sum(qty*(price-cost))

In [0]:
# Create a SparkSession
spark = SparkSession.builder.appName("ProductProfit").getOrCreate()
 
# Register the DataFrames as temporary tables to use in SQL queries
df_prod.createOrReplaceTempView("df_prod")
df_sales_ord.createOrReplaceTempView("df_sales_ord")
 
# Write the SQL query using backticks to reference column names with spaces
q = """SELECT p.`Product Name`, SUM(s.`Order Quantity` * (s.`Unit Price` - s.`Unit Cost`)) AS TotalProfit
FROM df_prod p
JOIN df_sales_ord s
ON p.`_ProductID` = s.`_ProductID`
GROUP BY p.`Product Name`
ORDER BY TotalProfit DESC
LIMIT 10
"""
 
# Execute the SQL query
result = spark.sql(q)
 
# Show the result
result.show()

+------------------+------------------+
|      Product Name|       TotalProfit|
+------------------+------------------+
|          Pendants|55020.759999999995|
|       Accessories| 52586.45000000001|
|            Phones|50823.050000000025|
|          Platters|          46374.19|
|  Dining Furniture|44116.159999999996|
|       Table Lamps|43996.420000000006|
|   Home Fragrances|          42046.68|
|  Cocktail Glasses|          41566.85|
|          Bakeware|41455.780000000006|
|Bathroom Furniture|          41103.11|
+------------------+------------------+



In [0]:
df_prod.join(df_sales_ord, df_prod["_ProductID"] == df_sales_ord["_ProductID"])\
         .groupBy(df_prod["Product Name"])\
         .agg(F.sum(df_sales_ord["Order Quantity"]*(df_sales_ord["Unit Price"] - df_sales_ord["Unit Cost"])).alias("Total_Profit"))\
         .orderBy(F.desc("Total_Profit"))\
         .show(10)

+------------------+------------------+
|      Product Name|      Total_Profit|
+------------------+------------------+
|          Pendants|55020.759999999995|
|       Accessories| 52586.45000000001|
|            Phones|50823.050000000025|
|          Platters|          46374.19|
|  Dining Furniture|44116.159999999996|
|       Table Lamps|43996.420000000006|
|   Home Fragrances|          42046.68|
|  Cocktail Glasses|          41566.85|
|          Bakeware|41455.780000000006|
|Bathroom Furniture|          41103.11|
+------------------+------------------+
only showing top 10 rows



## 4. Give top-3 stores selling product product number 25

In [0]:
# Create a SparkSession
spark = SparkSession.builder.appName("TotalSale").getOrCreate()
 
# Register the DataFrames as temporary tables to use in SQL queries
df_sales_ord.createOrReplaceTempView("df_sales_ord")
df_loc.createOrReplaceTempView("df_loc")
 
# Write the SQL query using backticks to reference column names with spaces
q = """
SELECT sl.`_StoreID`, sl.`City Name`, SUM(so.`Order Quantity`) AS TotalSale
FROM df_sales_ord so
JOIN df_loc sl ON so.`_StoreID` = sl.`_StoreID`
WHERE so.`_ProductID` = 25
GROUP BY sl.`_StoreID`, sl.`City Name`
ORDER BY TotalSale DESC, sl.`_StoreID` ASC
LIMIT 3
"""
 
# Execute the SQL query
result = spark.sql(q)
 
# Show the result
result.show()

+--------+----------+---------+
|_StoreID| City Name|TotalSale|
+--------+----------+---------+
|      56|    Orange|     16.0|
|      26|Costa Mesa|     14.0|
|     188|     Perry|     13.0|
+--------+----------+---------+



In [0]:
df_sales_ord.join(df_loc, df_sales_ord["_StoreID"] == df_loc["_StoreID"])\
            .where(df_sales_ord["_ProductID"] == 25)\
            .groupBy(df_loc["_StoreID"],"City Name")\
            .agg(F.sum(df_sales_ord["Order Quantity"]).alias("Total_Sale"))\
            .orderBy(F.desc("Total_Sale"), F.asc(df_loc["_StoreID"]))\
            .show(3)

+--------+----------+----------+
|_StoreID| City Name|Total_Sale|
+--------+----------+----------+
|      56|    Orange|      16.0|
|      26|Costa Mesa|      14.0|
|     188|     Perry|      13.0|
+--------+----------+----------+
only showing top 3 rows



## 5. Give top-3 products sold in midwest region

In [0]:
 # Create a SparkSession
spark = SparkSession.builder.appName("MidwestTopSale").getOrCreate()
 
# Register the DataFrames as temporary tables to use in SQL queries
df_reg.createOrReplaceTempView("df_reg")
df_loc.createOrReplaceTempView("df_loc")
df_sales_ord.createOrReplaceTempView("df_sales_ord")
df_prod.createOrReplaceTempView("df_prod")
 
# Write the SQL query using backticks to reference column names with spaces
q = """
SELECT p.`_ProductID`, p.`Product Name`, SUM(so.`Order Quantity`) AS TopSale
FROM df_reg r
JOIN df_loc sl ON r.`StateCode` = sl.`StateCode`
JOIN df_sales_ord so ON sl.`_StoreID` = so.`_StoreID`
JOIN df_prod p ON p.`_ProductID` = so.`_ProductID`
WHERE r.`Region` = "Midwest"
GROUP BY p.`_ProductID`, p.`Product Name`
ORDER BY TopSale DESC
LIMIT 3
"""
 
# Execute the SQL query
result = spark.sql(q)
 
# Show the result
result.show()

+----------+------------+-------+
|_ProductID|Product Name|TopSale|
+----------+------------+-------+
|        25|TV and video|  224.0|
|        23| Accessories|  206.0|
|        29|    Pendants|  206.0|
+----------+------------+-------+



In [0]:
df_reg.where(df_reg["Region"] == "Midwest")\
           .join(df_loc, df_reg["StateCode"] == df_loc["StateCode"])\
           .join(df_sales_ord, df_loc["_StoreID"] == df_sales_ord["_StoreID"])\
           .join(df_prod, df_prod["_ProductID"] == df_sales_ord["_ProductID"])\
           .groupBy(df_prod["_ProductID"], df_prod["Product Name"])\
           .agg(F.sum(df_sales_ord["Order Quantity"]).alias("TopSale"))\
           .orderBy(F.desc("TopSale"))\
           .show(3)

+----------+------------+-------+
|_ProductID|Product Name|TopSale|
+----------+------------+-------+
|        25|TV and video|  224.0|
|        29|    Pendants|  206.0|
|        23| Accessories|  206.0|
+----------+------------+-------+
only showing top 3 rows



## 6. Give region wise quantity sold for each product. Compute: Region, Product ID, Sum(Qty)

In [0]:
# Create a SparkSession
spark = SparkSession.builder.appName("TotalQuantitySoldByRegion").getOrCreate()
 
# Register the DataFrames as temporary tables to use in SQL queries
df_sales_team.createOrReplaceTempView("df_sales_team")
df_sales_ord.createOrReplaceTempView("df_sales_ord")
 
# Write the SQL query using backticks to reference column names with spaces
q = """SELECT st.`Region`, so.`_ProductID`, SUM(so.`Order Quantity`) AS TotalQuant
FROM df_sales_team st
JOIN df_sales_ord so ON st.`_SalesTeamID` = so.`_SalesTeamID`
GROUP BY st.`Region`, so.`_ProductID`
ORDER BY st.`Region`, so.`_ProductID`"""
 
# Execute the SQL query
result = spark.sql(q)
 
# Show the result
result.show()

+-------+----------+----------+
| Region|_ProductID|TotalQuant|
+-------+----------+----------+
|Midwest|         1|     192.0|
|Midwest|        10|     207.0|
|Midwest|        11|     233.0|
|Midwest|        12|     218.0|
|Midwest|        13|     194.0|
|Midwest|        14|     202.0|
|Midwest|        15|     164.0|
|Midwest|        16|     241.0|
|Midwest|        17|     192.0|
|Midwest|        18|     211.0|
|Midwest|        19|     209.0|
|Midwest|         2|     212.0|
|Midwest|        20|     224.0|
|Midwest|        21|     184.0|
|Midwest|        22|     247.0|
|Midwest|        23|     274.0|
|Midwest|        24|     213.0|
|Midwest|        25|     246.0|
|Midwest|        26|     192.0|
|Midwest|        27|     276.0|
+-------+----------+----------+
only showing top 20 rows



In [0]:
df_sales_team.join(df_sales_ord, df_sales_ord["_SalesTeamID"] == df_sales_team["_SalesTeamID"])\
           .groupBy(df_sales_team["Region"], df_sales_ord["_ProductID"])\
           .agg(F.sum(df_sales_ord["Order Quantity"]).alias("TotalQuant"))\
           .orderBy(df_sales_team["Region"], df_sales_ord["_ProductID"])\
           .show()

+-------+----------+----------+
| Region|_ProductID|TotalQuant|
+-------+----------+----------+
|Midwest|         1|     192.0|
|Midwest|        10|     207.0|
|Midwest|        11|     233.0|
|Midwest|        12|     218.0|
|Midwest|        13|     194.0|
|Midwest|        14|     202.0|
|Midwest|        15|     164.0|
|Midwest|        16|     241.0|
|Midwest|        17|     192.0|
|Midwest|        18|     211.0|
|Midwest|        19|     209.0|
|Midwest|         2|     212.0|
|Midwest|        20|     224.0|
|Midwest|        21|     184.0|
|Midwest|        22|     247.0|
|Midwest|        23|     274.0|
|Midwest|        24|     213.0|
|Midwest|        25|     246.0|
|Midwest|        26|     192.0|
|Midwest|        27|     276.0|
+-------+----------+----------+
only showing top 20 rows



## 7. Compute Average monthly sale in terms of numbers at each store; that , that is on average what numbers of a product are sold on a store in a month

In [0]:
# Create a SparkSession
spark = SparkSession.builder.appName("AverageMonthlySale").getOrCreate()
 
# Register the DataFrame as a temporary table to use in SQL queries
df_sales_ord.createOrReplaceTempView("df_sales_ord")
 
# Write the SQL query to calculate average monthly sale
q = """
SELECT s.`_StoreID`, 
       AVG(s.`MonthlySale`) AS AvgMonthlySale
FROM (
    SELECT `_StoreID`, 
           SUBSTR(`OrderDate`, 4, 7) AS `Month-Year`, 
           SUM(`Order Quantity`) AS `MonthlySale`
    FROM df_sales_ord
    GROUP BY `_StoreID`, `Month-Year`
) s
GROUP BY s.`_StoreID`
ORDER BY s.`_StoreID`
"""
 
# Execute the SQL query
result = spark.sql(q)
 
# Show the result
result.show(367)

+--------+------------------+
|_StoreID|    AvgMonthlySale|
+--------+------------------+
|       1| 6.421052631578948|
|      10|               4.5|
|     100| 7.714285714285714|
|     101| 5.222222222222222|
|     102| 6.421052631578948|
|     103|4.7894736842105265|
|     104| 4.444444444444445|
|     105|               4.5|
|     106|              4.05|
|     107|               6.8|
|     108| 4.461538461538462|
|     109| 3.888888888888889|
|      11|5.2727272727272725|
|     110| 6.684210526315789|
|     111|6.3076923076923075|
|     112|             5.625|
|     113| 6.666666666666667|
|     114|            5.3125|
|     115| 6.214285714285714|
|     116| 5.722222222222222|
|     117| 5.153846153846154|
|     118| 5.111111111111111|
|     119| 5.722222222222222|
|      12| 6.684210526315789|
|     120| 5.833333333333333|
|     121|5.7727272727272725|
|     122|               6.4|
|     123| 5.166666666666667|
|     124| 6.071428571428571|
|     125| 9.466666666666667|
|     126|

In [0]:
# def get_month(date):
#     return date[3:]
 
monthUDF = F.udf(lambda z: z[3:])
 
df_sales_ord.groupBy(df_sales_ord["_StoreID"], monthUDF(F.col("OrderDate")).alias("MonthYear"))\
            .agg(F.sum(df_sales_ord["Order Quantity"]).alias("MonthlySale"))\
            .groupBy(df_sales_ord["_StoreID"])\
            .agg(F.avg("MonthlySale").alias("AvgMonthlySale"))\
            .orderBy(df_sales_ord["_StoreID"])\
            .show(367)

+--------+------------------+
|_StoreID|    AvgMonthlySale|
+--------+------------------+
|       1| 6.421052631578948|
|      10|               4.5|
|     100| 7.714285714285714|
|     101| 5.222222222222222|
|     102| 6.421052631578948|
|     103|4.7894736842105265|
|     104| 4.444444444444445|
|     105|               4.5|
|     106|              4.05|
|     107|               6.8|
|     108| 4.461538461538462|
|     109| 3.888888888888889|
|      11|5.2727272727272725|
|     110| 6.684210526315789|
|     111|6.3076923076923075|
|     112|             5.625|
|     113| 6.666666666666667|
|     114|            5.3125|
|     115| 6.214285714285714|
|     116| 5.722222222222222|
|     117| 5.153846153846154|
|     118| 5.111111111111111|
|     119| 5.722222222222222|
|      12| 6.684210526315789|
|     120| 5.833333333333333|
|     121|5.7727272727272725|
|     122|               6.4|
|     123| 5.166666666666667|
|     124| 6.071428571428571|
|     125| 9.466666666666667|
|     126|

## 8. Compute sales bifurcation of each warehouse; that total sales amount through each channel

In [0]:
from pyspark.sql import SparkSession
 
# Create a SparkSession
spark = SparkSession.builder.appName("TotalSalesThroughChannel").getOrCreate()
 
# Register the DataFrame as a temporary table to use in SQL queries
df_sales_ord.createOrReplaceTempView("df_sales_ord")
 
# Write the SQL query to calculate total sales by warehouse and sales channel
q = """
SELECT `WarehouseCode`, `Sales Channel`, SUM(`Total_Sales`) AS `Total_Sales`
FROM (
    SELECT `WarehouseCode`, `Sales Channel`, (`Order Quantity` * `Unit Price`) AS `Total_Sales`
    FROM df_sales_ord
) subquery
GROUP BY `WarehouseCode`, `Sales Channel`
ORDER BY `WarehouseCode`, `Sales Channel`
"""
 
# Execute the SQL query
result = spark.sql(q)
 
# Show the result
result.show()

+-------------+-------------+------------------+
|WarehouseCode|Sales Channel|       Total_Sales|
+-------------+-------------+------------------+
| WARE-MKL1006|  Distributor| 99850.10000000002|
| WARE-MKL1006|     In-Store|          211224.2|
| WARE-MKL1006|       Online|          142086.9|
| WARE-MKL1006|    Wholesale|           19208.9|
| WARE-NBV1002|  Distributor|           53191.3|
| WARE-NBV1002|     In-Store|181275.19999999998|
| WARE-NBV1002|       Online|121323.59999999998|
| WARE-NBV1002|    Wholesale|           50712.3|
| WARE-NMK1003|  Distributor|228081.40000000002|
| WARE-NMK1003|     In-Store| 578055.9000000001|
| WARE-NMK1003|       Online| 541125.5000000002|
| WARE-NMK1003|    Wholesale|130817.49999999999|
| WARE-PUJ1005|  Distributor|135380.19999999998|
| WARE-PUJ1005|     In-Store|          297285.7|
| WARE-PUJ1005|       Online|243002.30000000002|
| WARE-PUJ1005|    Wholesale|104754.49999999999|
| WARE-UHY1004|  Distributor|          130529.4|
| WARE-UHY1004|     

In [0]:
df_sales_ord.groupBy(df_sales_ord["WarehouseCode"], df_sales_ord["Sales Channel"])\
           .agg(F.sum(df_sales_ord["Order Quantity"]*df_sales_ord["Unit Price"]).alias("Total_Sales"))\
           .orderBy(df_sales_ord["WarehouseCode"], df_sales_ord["Sales Channel"])\
           .show()

+-------------+-------------+------------------+
|WarehouseCode|Sales Channel|       Total_Sales|
+-------------+-------------+------------------+
| WARE-MKL1006|  Distributor| 99850.10000000002|
| WARE-MKL1006|     In-Store|          211224.2|
| WARE-MKL1006|       Online|          142086.9|
| WARE-MKL1006|    Wholesale|           19208.9|
| WARE-NBV1002|  Distributor|           53191.3|
| WARE-NBV1002|     In-Store|181275.19999999998|
| WARE-NBV1002|       Online|121323.59999999998|
| WARE-NBV1002|    Wholesale|           50712.3|
| WARE-NMK1003|  Distributor|228081.40000000002|
| WARE-NMK1003|     In-Store| 578055.9000000001|
| WARE-NMK1003|       Online| 541125.5000000002|
| WARE-NMK1003|    Wholesale|130817.49999999999|
| WARE-PUJ1005|  Distributor|135380.19999999998|
| WARE-PUJ1005|     In-Store|          297285.7|
| WARE-PUJ1005|       Online|243002.30000000002|
| WARE-PUJ1005|    Wholesale|104754.49999999999|
| WARE-UHY1004|  Distributor|          130529.4|
| WARE-UHY1004|     

## 9. Compute average "product retention period" (i. e. the difference between procurement date and order date) at each warehouse

In [0]:
from pyspark.sql.types import IntegerType
 
def get_retention(procure, order):
    from datetime import datetime
    procure_date = datetime.strptime(procure, "%m/%d/%Y")
    order_date = datetime.strptime(order, "%m/%d/%Y")
    return (order_date - procure_date).days
 
retentionUDF = F.udf(lambda x, y: get_retention(x, y), IntegerType())
 
result_df = df_sales_ord.withColumn("Retention", retentionUDF(F.col("ProcuredDate"), F.col("OrderDate"))) \
    .groupBy("WarehouseCode") \
    .agg(F.avg("Retention").alias("AverageRetention(days)"))
 
result_df.show()

+-------------+----------------------+
|WarehouseCode|AverageRetention(days)|
+-------------+----------------------+
| WARE-XYS1001|    109.35679214402619|
| WARE-PUJ1005|    109.51274982770504|
| WARE-MKL1006|     108.2532088681447|
| WARE-NMK1003|    109.46986027944112|
| WARE-UHY1004|    108.73359683794466|
| WARE-NBV1002|    109.81476121562952|
+-------------+----------------------+



## 10. Give Year-Month sale of all products

In [0]:
from pyspark.sql.functions import date_format, col, to_date, datediff, avg, expr
 
# Create a SparkSession
spark = SparkSession.builder.appName("YearMonthSale").getOrCreate()
 
# Extract the day, month, and year components
df = df_sales_ord.withColumn("Month", expr("month(to_date(OrderDate, 'M/d/yyyy'))"))
df = df_sales_ord.withColumn("Day", expr("day(to_date(OrderDate, 'M/d/yyyy'))"))
df = df_sales_ord.withColumn("Year", expr("year(to_date(OrderDate, 'M/d/yyyy'))"))
 
# Format the date as "MM/DD/YYYY"
df = df_sales_ord.withColumn("OrderDate", date_format(to_date(col("OrderDate"), "M/d/yyyy"), "MM/dd/yyyy"))
 
def get_month(date):
    return date[6:]+"-"+date[0:2]
 
monthUDF = F.udf(lambda z: get_month(z))
 
df.join(df_prod, df["_ProductID"] == df_prod["_ProductID"])\
            .groupBy(df["_ProductID"], df_prod["Product Name"], monthUDF(F.col("OrderDate")).alias("Year-Month"))\
            .agg(F.sum(df_sales_ord["Order Quantity"]).alias("SaleInMonth"))\
            .orderBy(df["_ProductID"], df_prod["Product Name"], "Year-Month")\
            .show()

+----------+------------+----------+-----------+
|_ProductID|Product Name|Year-Month|SaleInMonth|
+----------+------------+----------+-----------+
|         1|    Cookware|   2018-05|        5.0|
|         1|    Cookware|   2018-06|       24.0|
|         1|    Cookware|   2018-07|       39.0|
|         1|    Cookware|   2018-08|       22.0|
|         1|    Cookware|   2018-09|       21.0|
|         1|    Cookware|   2018-10|       30.0|
|         1|    Cookware|   2018-11|       23.0|
|         1|    Cookware|   2018-12|       30.0|
|         1|    Cookware|   2019-01|       15.0|
|         1|    Cookware|   2019-02|       10.0|
|         1|    Cookware|   2019-03|       19.0|
|         1|    Cookware|   2019-04|       18.0|
|         1|    Cookware|   2019-05|       36.0|
|         1|    Cookware|   2019-06|       21.0|
|         1|    Cookware|   2019-07|       21.0|
|         1|    Cookware|   2019-08|       10.0|
|         1|    Cookware|   2019-09|       35.0|
|         1|    Cook

## 11. Compute a fact file with the dimensions of " store_id ", " product_id ", " month_year "

In [0]:
df_sales_ord.select(F.col("_StoreID").alias("StoreID"),\
                   F.col("_ProductID").alias("ProductID"),\
                   F.col("OrderDate").alias("Date"),
                   F.col("Order Quantity").alias("Quantity"),
                   F.col("Unit Price").alias("Price"))\
            .cube("StoreID","ProductID","Date")\
            .agg(F.sum("Quantity").alias("Quantity"), F.sum(F.col("Quantity") * F.col("Price")).alias("Amount"))\
            .orderBy(F.asc_nulls_last("StoreID"), F.asc_nulls_last("ProductID"))\
            .show(100)

+-------+---------+----------+--------+-----------------+
|StoreID|ProductID|      Date|Quantity|           Amount|
+-------+---------+----------+--------+-----------------+
|      1|       11|      null|     5.0|             null|
|      1|       11|11/17/2019|     5.0|             null|
|      1|       16| 9/13/2020|     6.0|             null|
|      1|       16|  8/6/2019|     7.0|           6237.7|
|      1|       16|      null|    13.0|           6237.7|
|      1|       19|  6/4/2020|     8.0|           1715.2|
|      1|       19|      null|     8.0|           1715.2|
|      1|       22| 3/28/2020|     6.0|           1407.0|
|      1|       22| 4/27/2019|     6.0|5467.200000000001|
|      1|       22|      null|    12.0|6874.200000000001|
|      1|       23|      null|     8.0|             null|
|      1|       23| 8/21/2020|     8.0|             null|
|      1|       27|      null|     4.0|             null|
|      1|       27| 2/20/2020|     4.0|             null|
|      1|     